In [ ]:
import os
import pprint
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
model_data = pd.read_csv('model_data.csv')

In [ ]:
model_data = model_data.set_index(pd.to_datetime(model_data['index'], unit='ns'))

In [ ]:
model_data.drop(columns='index', inplace=True)

In [ ]:
model_data

In [ ]:
for i in model_data.columns:
    print(i, model_data[i].min(), model_data[i].max())

In [ ]:
depth_diff = model_data.depth_hbc.abs() - model_data.V_Depth.abs()
depth_diff.min(), depth_diff.max()

In [ ]:
model_data = model_data.loc[~model_data.index.duplicated(), :]

In [ ]:
sns.pairplot(model_data[["speed", "Water_Depth", "Altimeter"]], diag_kind="kde")

In [ ]:
for i in model_data.columns:
    print(i, model_data[i].min(), model_data[i].max())

## Filtering data
* HBC at the seafloor: `Water_Depth: [40, 120]` $m$
* Vessel speed is in the range `[2, 7]` $kn$ 
* Distance between Vessel and HBC less than a threshold  `[0, 300]` $m$
* HBS distance from seafloor in the range `[0, 5]` $m$

In [ ]:
model_data_test = model_data.copy()

model_data_test = model_data_test[
    (model_data_test.Water_Depth >= 40) & (model_data_test.Water_Depth <= 120)
]

model_data_test = model_data_test[
    (model_data_test.Altimeter >= 0) & (model_data_test.Altimeter <= 4)
]

model_data_test = model_data_test[
    (model_data_test.speed >= 2) & (model_data_test.speed <= 5)
]

model_data_test = model_data_test[
    (model_data_test.distance >= 0) & (model_data_test.distance <= 300)
]

In [ ]:
model_data_test.shape

In [ ]:
sns.pairplot(model_data_test[["speed", "Water_Depth", "Altimeter"]], diag_kind="kde")

In [ ]:
model_data_test = model_data_test[model_data_test.V_Depth >= 30]

In [ ]:
sns.pairplot(model_data_test[["speed", "Water_Depth", "Altimeter"]], diag_kind="kde")

# Regression

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras

In [ ]:
data = model_data_test[["speed", "Water_Depth", "Altimeter"]].values[1:]

In [ ]:
model_data_test_stats = model_data_test.describe()
# model_data_test_stats.pop("")
model_data_test_stats = model_data_test_stats.transpose()
model_data_test_stats

In [ ]:
data = model_data_test[["speed", "Water_Depth", "Altimeter"]].values[1:]

In [ ]:
#data = model_data_test[["speed", "Water_Depth"]].values[1:]

In [ ]:
data = model_data_test[["speed", "Altimeter"]].values[1:]

In [ ]:
data.shape

In [ ]:
target = model_data_test[["distance"]].values[1:]

In [ ]:
target = target / target.max()

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    data, target, random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
scaler = StandardScaler()
# scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
# !pip install git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
model = keras.models.Sequential(
    [
        keras.layers.Dense(
            64,
            activation="relu",
            input_shape=X_train.shape[1:],
            kernel_regularizer=keras.regularizers.l2(0.01),
            activity_regularizer=keras.regularizers.l1(0.01),
        ),
        keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
        keras.layers.Dense(4, activation="relu"),
        keras.layers.Dense(1, activation="linear"),  # sigmoid, linear
    ]
)

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)

In [ ]:
lr = 1e-4
decay = 1e-6

opt = keras.optimizers.Adam(learning_rate=lr)

In [ ]:
model.compile(loss="mean_squared_error", optimizer=opt, metrics=["mae", "mse"])

In [ ]:
epochs = 1000

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stop, tfdocs.modeling.EpochDots()],
)

In [ ]:
mse_test = model.evaluate(X_test, y_test)

In [ ]:
mse_test

In [ ]:
X_new = X_test
y_pred = model.predict(X_new)

In [ ]:
y_pred

In [ ]:
rescale = model_data_test[["distance"]].values[1:].max()

In [ ]:
# rescale

In [ ]:
y_pred * rescale

In [ ]:
y_test * rescale

In [ ]:
err = (y_pred * rescale) - (y_test * rescale)

In [ ]:
err

In [ ]:
err.min(), err.max(), err.mean()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({"Basic": history}, metric="mse")

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({"Basic": history}, metric="mae")

In [ ]:
_ = plt.hist(err, bins="auto")

In [ ]:
model.save("hbc_layback.h5")

In [ ]:
rescale

In [ ]:
X_new